In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
pip install trimesh

In [ ]:
import os
import sys

Upload ảnh query vào Sino-Nom 2D-3D Retrieval/input/2d_images', stl file vào input/3d_meshes.

In [ ]:
root_path = '/content/gdrive/MyDrive/Sino-Nom 2D-3D Retrieval/'
if 'Sino-Nom 2D-3D Retrieval' not in os.listdir('/content/gdrive/MyDrive/'):
    os.mkdir('/content/gdrive/MyDrive/Sino-Nom 2D-3D Retrieval')
    os.mkdir('/content/gdrive/MyDrive/Sino-Nom 2D-3D Retrieval/input')
    os.mkdir('/content/gdrive/MyDrive/Sino-Nom 2D-3D Retrieval/input/2d_images')
    os.mkdir('/content/gdrive/MyDrive/Sino-Nom 2D-3D Retrieval/input/3d_meshes')
    os.mkdir('/content/gdrive/MyDrive/Sino-Nom 2D-3D Retrieval/output')
    os.mkdir('/content/gdrive/MyDrive/Sino-Nom 2D-3D Retrieval/output/processed_images')
    os.mkdir('/content/gdrive/MyDrive/Sino-Nom 2D-3D Retrieval/output/2d_projections')

In [ ]:
def extract_filename(filename):
  return int(filename[:-4])

In [ ]:
extract_filename('123.stl')

123

In [ ]:
_2d_images = sorted(os.listdir('/content/gdrive/MyDrive/Sino-Nom 2D-3D Retrieval/input/2d_images'), key = extract_filename)

NameError: name 'os' is not defined

In [ ]:
_3d_meshes = sorted(os.listdir('/content/gdrive/MyDrive/Sino-Nom 2D-3D Retrieval/input/3d_meshes'), key = extract_filename)

In [ ]:
_2d_images

Data loader

In [ ]:
import numpy as np
import cv2
import trimesh
import matplotlib.pyplot as plt

In [ ]:
def binary_image(img, thres_hold = 150):
  return cv2.threshold(img, thres_hold, 255, cv2.THRESH_BINARY)[1]

In [ ]:
def trimming_white(binary_img, output_size = (512, 512)):
  non_white_pixels = np.where(binary_img != 255)

  # Find the bounding points of non-white areas.
  min_x = np.min(non_white_pixels[1])
  max_x = np.max(non_white_pixels[1])
  min_y = np.min(non_white_pixels[0])
  max_y = np.max(non_white_pixels[0])

  # Trim the image to remove the white
  trimmed_image = cv2.resize(binary_img[min_y:max_y+1, min_x:max_x+1], output_size)

  return trimmed_image

In [ ]:
def convert_to_2d(file_path):
    # Read STL file
    mesh = trimesh.load(file_path)

    # Counting bounding box
    min_coords = np.min(mesh.vertices, axis=0)
    max_coords = np.max(mesh.vertices, axis=0)

    # Chop the 3D mesh into slices from the top down.
    z_min, z_max = min_coords[2], max_coords[2]
    z_step = (z_max - z_min) / 8
    fig, ax = plt.subplots(figsize=(2,2), dpi=64)
    ax.axis('off')

    # Take out the first slice
    cutting_plane = z_max - z_step

    # Get faces are above the cutting_plane
    above_cutting_plane_faces = []
    for face in mesh.faces:
        if np.all(mesh.vertices[face, 2] > cutting_plane):
            above_cutting_plane_faces.append(face)

    # Create new mesh from faces_above_cutting_plane
    above_cutting_plane_mesh = trimesh.Trimesh(vertices=mesh.vertices, faces=above_cutting_plane_faces)

    # Draw the flat projection
    ax.scatter(above_cutting_plane_mesh.vertices[:, 0], above_cutting_plane_mesh.vertices[:, 1], s=1, alpha=0.5)

    fig.canvas.draw()
    img_data = np.frombuffer(fig.canvas.tostring_rgb(), dtype=np.uint8)
    img_data = img_data.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    img = cv2.cvtColor(img_data, cv2.COLOR_RGB2GRAY)
    img = cv2.resize(img, (1024, 1024))
    img = trimming_white(binary_image(img, 150))
    return img

In [ ]:
for file_name in _2d_images:
  img = cv2.imread('test_retrieval/query/' + file_name, cv2.IMREAD_GRAYSCALE)
  img = trimming_white(binary_image(img, 150))
  cv2.imwrite(root_path + 'output/processed_images/' + file_name, img)

In [ ]:
for file in _3d_meshes:
  img_2d = convert_to_2d('test_retrieval/database/'+ file)
  cv2.imwrite(root_path + 'output/2d_projections/' + file[:-3] + "png", img_2d)